In [1]:
import os
import sys
os.chdir('..')
sys.path.append(os.getcwd())

In [2]:
from Models.pretrain import *
from Models.cPB import cPB
import torch.nn.functional as F
import pandas as pd
from Models.cGRU_Model import cGRU
import pickle
import warnings

warnings.filterwarnings("ignore", category=FutureWarning, module='torch.storage')
warnings.filterwarnings("ignore", category=UserWarning, module='torch.storage')
warnings.filterwarnings("ignore", category=DeprecationWarning, module='torch.storage')

# SINE datasets

## Executable code:


In [ ]:
for iter in range(1, 11):
    ###### Config the parameters ###### 
    base_model = 'GRU'
    batch_size = 128
    hidden_size = 50
    seq_len = 10
    epoch_size = 10
    lr = 0.01
    number_of_tasks = 4
    input_size = 2
    iteration = iter
    dataset = 'SINE'
    dataset_name = "sine_rw10_mode5_extended_16-16_1234"
    Pretrain_task = '1'
    output_size = 2
    ###### End of Config ######
    df = pd.read_csv(os.path.join(f"datasets/{dataset}/", f"{dataset_name}.csv"))
    pretrain_model_addr = f'Performance/Pretrain/{base_model}/{dataset}/Before/{dataset}-Task_{Pretrain_task}-{base_model}-pretrain-hidden{hidden_size}-epoch10_iter{iteration}.pickle'

    # Initialize cGRU model
    model = cGRU(lr=lr, hidden_size=hidden_size, seq_len=seq_len, 
                 pretrain_model_addr=pretrain_model_addr, input_size=input_size, 
                 epoch_size=epoch_size, batch_first=True, output_size = output_size)


    for task in range(1, df["task"].max() + 1):
        df_task = df[df["task"] == task]
        df_task = df_task.drop(columns="task")
        batch_cont = 0

        for i in range(0, len(df_task), batch_size):
            x = df_task.iloc[i: i + batch_size, :-1].values.astype(np.float32)
            y = list(df_task.iloc[i: i + batch_size, -1])

            model.predict_many(x, y, task_number=task)
            model.learn_many(x, y)
            batch_cont += 1
            

        model.save_final_metrics(task)

    file_path = f'Performance/Results/cGRU/Periodic/{dataset}/pretrain_T{Pretrain_task}_{base_model}-{dataset_name}-hidden{hidden_size}-epoch10-iter{iteration}_periodic.pkl'
    with open(file_path, 'wb') as file:
        pickle.dump(model.performance, file)
        
    with open(f"Performance/Final Models/cGRU/{base_model}/{dataset}/Periodic/{dataset_name}-{base_model}-hidden{hidden_size}-epoch10-iter{iteration}.pickle", "wb") as fp:
        pickle.dump(model.model.state_dict(), fp)


    print(f'Iteration {iteration} finished')


# Weather datasets

## Executable code:


In [ ]:
for iter in range(1, 11):
    ###### Config the parameters ###### 
    base_model = 'GRU'
    batch_size = 128
    hidden_size = 250
    seq_len = 11
    epoch_size = 10
    lr = 0.01
    number_of_tasks = 4
    mask_selection_NofBatch = 50
    input_size = 4
    iteration = iter
    dataset = 'Weather'
    dataset_name = "weather_st124_1conf"
    Pretrain_task = '1'
    output_size = 2
    ####### End of Config ######
    df = pd.read_csv(os.path.join(f"datasets/{dataset}/", f"{dataset_name}.csv"))
    pretrain_model_addr = f'Performance/Pretrain/{base_model}/{dataset}/Before/{dataset}-{base_model}-pretrain-hidden{hidden_size}-epoch10_itter{iteration}.pickle'

    # Initialize cGRU model
    model = cGRU(lr=lr, hidden_size=hidden_size, seq_len=seq_len, 
                 pretrain_model_addr=pretrain_model_addr, input_size=input_size, 
                 epoch_size=epoch_size, batch_first=True, output_size = output_size)


    for task in range(1, df["task"].max() + 1):
        df_task = df[df["task"] == task]
        df_task = df_task.drop(columns="task")
        batch_cont = 0

        for i in range(0, len(df_task), batch_size):
            x = df_task.iloc[i: i + batch_size, :-1].values.astype(np.float32)
            y = list(df_task.iloc[i: i + batch_size, -1])

            model.predict_many(x, y, task_number=task)
            model.learn_many(x, y)
            batch_cont += 1

        model.save_final_metrics(task)

    file_path = f'Performance/Results/cGRU/Periodic/{dataset}/pretrain_T{Pretrain_task}_{base_model}-{dataset_name}-hidden{hidden_size}-epoch10-iter{iteration}_periodic.pkl'
    with open(file_path, 'wb') as file:
        pickle.dump(model.performance, file)
        
    with open(f"Performance/Final Models/cGRU/{base_model}/{dataset}/Periodic/{dataset_name}-{base_model}-hidden{hidden_size}-epoch10-iter{iteration}.pickle", "wb") as fp:
        pickle.dump(model.model.state_dict(), fp)

    print(f'Iteration {iteration} finished')
